In [2]:
import os
from os.path import join
import re
import math
import pickle as pkl
import time
import numpy as np
import pandas as pd
from easydict import EasyDict as edict
import h5py
import seaborn as sns
import matplotlib.pyplot as plt
from neuro_data_analysis.mat_data_translate_lib import h5_to_dict_simplify, print_hdf5_info
# set the pandas display options width
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)


mat_root = r"S:\Data-Ephys-MAT"
pkl_root = r"S:\Data-Ephys-PKL"
exp_record_pathdict = {"Alfa": r"S:\Exp_Record_Alfa.xlsx", 
                       "Beto": r"S:\ExpSpecTable_Augment.xlsx",
                       "Caos": r"S:\Exp_Record_Caos.xlsx",
                       "Diablito": r"S:\Exp_Record_Diablito.xlsx"}

ExpRecord_CD = pd.concat([pd.read_excel(exp_record_pathdict[Animal]) for Animal in ("Caos", "Diablito")])
exp_mask = ExpRecord_CD.Exp_collection.str.contains('BigGAN_Hessian', na=False) & ~ ExpRecord_CD.Expi.isna()
ExpRecord_Hessian = ExpRecord_CD.loc[exp_mask, :]
exp_mask = ExpRecord_CD.Exp_collection.str.contains('BigGAN_FC6', na=False) & ~ ExpRecord_CD.Expi.isna()
ExpRecord_Evol = ExpRecord_CD.loc[exp_mask, :]
print(ExpRecord_Hessian)
print(ExpRecord_Evol)

    ChanQual  Exp_collection  Expi  WebCam                                           comments                ephysFN                              expControlFN  pref_chan  pref_unit  stim_size                                            stimuli
12       NaN  BigGAN_Hessian   1.0     NaN  Manifold with biggan\n 003 at 309 PM, manifold...      Caos-12022024-003      241202_150857_Caos_selectivity_basic       91.0        2.0        NaN  N:\Stimuli\Evolutions\2024\2024-12-02-Ev-Caos-...
15       NaN  BigGAN_Hessian   2.0     NaN   006 manifold\n alas, it's all dogs\n 006 at 3...      Caos-12022024-006      241202_155547_Caos_selectivity_basic       94.0        1.0        NaN  N:\Stimuli\Evolutions\2024\2024-12-02-Ev-Caos-...
18       NaN  BigGAN_Hessian   3.0     NaN  003 at 239 PM\nmanifold\n(0,0) 2\nhas had arou...      Caos-12042024-003      241204_143902_Caos_selectivity_basic       71.0        2.0        2.0  N:\Stimuli\Evolutions\2024\2024-12-04-Ev-Caos-...
22       NaN  BigGAN_Hessian

### Find the older experiments

In [3]:
ExpRecord_All = pd.concat([pd.read_excel(exp_record_pathdict[Animal]) for Animal in ("Caos", "Diablito", "Alfa", "Beto")])
exp_mask = ExpRecord_All.Exp_collection.str.contains('BigGAN_Hessian', na=False) & \
            ExpRecord_All.expControlFN.str.contains('selectivity', na=False) & \
                ExpRecord_All.Expi.notna()
ExpRecord_Hessian_All = ExpRecord_All.loc[exp_mask, :]          

In [5]:
print(ExpRecord_Hessian_All.shape)
ExpRecord_Hessian_All

(41, 11)


,ChanQual,Exp_collection,Expi,WebCam,comments,ephysFN,expControlFN,pref_chan,pref_unit,stim_size,stimuli
12,NaN,BigGAN_Hessian,1.0,NaN,"Manifold with biggan\n 003 at 309 PM, manifold...",Caos-12022024-003,241202_150857_Caos_selectivity_basic,91.0,2.0,NaN,N:\Stimuli\Evolutions\2024\2024-12-02-Ev-Caos-...
15,NaN,BigGAN_Hessian,2.0,NaN,"006 manifold\n alas, it's all dogs\n 006 at 3...",Caos-12022024-006,241202_155547_Caos_selectivity_basic,94.0,1.0,NaN,N:\Stimuli\Evolutions\2024\2024-12-02-Ev-Caos-...
18,NaN,BigGAN_Hessian,3.0,NaN,"003 at 239 PM\nmanifold\n(0,0) 2\nhas had arou...",Caos-12042024-003,241204_143902_Caos_selectivity_basic,71.0,2.0,2.0,N:\Stimuli\Evolutions\2024\2024-12-04-Ev-Caos-...
22,NaN,BigGAN_Hessian,4.0,NaN,Running manifold\n007 at 3:30 PM\nch 72 ( 0 0)...,Caos-12042024-007,241204_152944_Caos_selectivity_basic,72.0,1.0,NaN,N:\Stimuli\Evolutions\2024\2024-12-04-Ev-Caos-...
26,NaN,BigGAN_Hessian,5.0,NaN,Manifold selectivity basic started at 2.48PM -...,Caos-12092024-004,241209_144750_Caos_selectivity_basic,92.0,2.0,2.0,N:\Stimuli\Evolutions\2024\2024-12-09-Ev-Caos-...
29,NaN,BigGAN_Hessian,6.0,NaN,Manifold started at 1.41PM\nFilled up to 200cc...,Caos-12102024-003,241210_134049_Caos_selectivity_basic,68.0,2.0,2.0,N:\Stimuli\Evolutions\2024\2024-12-10-Ev-Caos-...
32,NaN,BigGAN_Hessian,7.0,NaN,"Manifold started 2.40PM\nMU on channel 69\n(0,...",Caos-12102024-006,241210_143915_Caos_selectivity_basic,69.0,1.0,2.0,N:\Stimuli\Evolutions\2024\2024-12-10-Ev-Caos-...
2,NaN,BigGAN_Hessian,1.0,NaN,"Running manifold\ntakes time, seems to hang on...",Diablito-03122024-003,241203_141151_Diablito_selectivity_basic,56.0,1.0,4.0,N:\Stimuli\Evolutions\2024\2024-12-03-Ev-Diabl...
7,NaN,BigGAN_Hessian,2.0,NaN,"003 manifold at 129 PM, (-1,-1) 4 stimulus pos...",Diablito-11122024-003,241211_132910_Diablito_selectivity_basic,42.0,2.0,4.0,N:\Stimuli\Evolutions\2024\2024-12-11-Ev-Diabl...
9,NaN,BigGAN_Hessian,3.0,NaN,Started selectivity basic 2.17PM\n34 (-2 0) 4 ...,Diablito-11122024-005,241211_141636_Diablito_selectivity_basic,34.0,2.0,4.0,N:\Stimuli\Evolutions\2024\2024-12-11-Ev-Diabl...


### Backup and sync the files 

In [12]:
local_mat_root = r"S:\Data-Ephys-MAT"
network_mat_root = r"N:\Data-Ephys-MAT"
file_df = []
for _, exprow in ExpRecord_Hessian_All.iterrows():
    # print(exprow.ephysFN,exprow.Expi)
    # check if the mat file exists
    net_raw_mat_path = join(network_mat_root, f"{exprow.ephysFN}.mat")
    net_formatted_mat_path = join(network_mat_root, f"{exprow.ephysFN}_formatted.mat")
    local_raw_mat_path = join(local_mat_root, f"{exprow.ephysFN}.mat")
    local_formatted_mat_path = join(local_mat_root, f"{exprow.ephysFN}_formatted.mat")
    exist_net_raw = os.path.exists(net_raw_mat_path)
    exist_net_formatted = os.path.exists(net_formatted_mat_path)
    exist_local_raw = os.path.exists(local_raw_mat_path)
    exist_local_formatted = os.path.exists(local_formatted_mat_path)
    # check the existence of the mat file, summarize the result in one line
    mat_exist_str = f"Net: Raw {exist_net_raw}, Formatted {exist_net_formatted}, Local: Raw {exist_local_raw}, Formatted {exist_local_formatted}"
    # print(mat_exist_str)
    # Policy: 
    #   if local exists any, copy the local raw / formatted file to the network folder
    #   if network exists formatted, copy the network formatted file to the local folder 
    #   if network exists raw, copy the network raw file to the local folder 
    
    file_df.append({"ephysFN": exprow.ephysFN, "exist_net_raw": exist_net_raw, "exist_net_formatted": exist_net_formatted, 
                              "exist_local_raw": exist_local_raw, "exist_local_formatted": exist_local_formatted})
file_df = pd.DataFrame(file_df)
file_df

,ephysFN,exist_net_raw,exist_net_formatted,exist_local_raw,exist_local_formatted
0,Caos-12022024-003,False,False,True,True
1,Caos-12022024-006,False,False,True,True
2,Caos-12042024-003,False,False,True,True
3,Caos-12042024-007,False,False,True,True
4,Caos-12092024-004,False,False,True,True
5,Caos-12102024-003,False,False,True,True
6,Caos-12102024-006,False,False,True,True
7,Diablito-03122024-003,False,False,True,True
8,Diablito-11122024-003,False,False,True,True
9,Diablito-11122024-005,False,False,True,True


In [15]:
from tqdm import tqdm
from neuro_data_analysis.mat_data_translate_lib import h5_to_dict_simplify, print_hdf5_info
mat_root = r"S:\Data-Ephys-MAT"
pkl_root = r"S:\Data-Ephys-PKL"
T0 = time.time()
for ephys_name in tqdm(ExpRecord_Hessian_All.ephysFN.values):
    if os.path.exists(join(pkl_root, f"{ephys_name}.pkl")):
        print(f"PKL file already exists for {ephys_name}")
        continue
    t0 = time.time()
    mat_file = join(mat_root, f"{ephys_name}_formatted.mat")
    data = h5py.File(mat_file, "r")
    print(f"Loading {ephys_name} mat file")
    t1 = time.time()
    print(f"Time taken: {t1 - t0:.2f} seconds for loading mat file")
    # fast selective loading
    meta_dict = h5_to_dict_simplify(data['meta'], data)
    # turn rasters into a numpy array, reduce precision to 32-bit float
    rasters = np.array(data['rasters']).astype(np.float32)
    # lfps = np.array(data['lfps']).astype(np.float32)
    Trial_dict = edict()
    for key in data['Trials'].keys():
        if key in ["B", "MLConfig"]:
            continue
        print(f"Loading {key}", end="\t")
        Trial_dict[key] = h5_to_dict_simplify(data['Trials'][key], data)
        print("done")
    # TrialRecord_dict = h5_to_dict_simplify(data['Trials/TrialRecord'], data)
    t2 = time.time()
    print(f"Time taken: {t2 - t1:.2f} seconds for loading trials")

    pkl.dump({"Trials": Trial_dict, "meta": meta_dict, "rasters": rasters, }, open(join(pkl_root, f"{ephys_name}.pkl"), "wb"))
    t3 = time.time()
    print(f"Time taken: {t3 - t2:.2f} seconds for dumping pkl file [Total: {t3 - T0:.2f} seconds]")


  0%|          | 0/41 [00:00<?, ?it/s]

PKL file already exists for Caos-12022024-003
PKL file already exists for Caos-12022024-006
PKL file already exists for Caos-12042024-003
PKL file already exists for Caos-12042024-007
PKL file already exists for Caos-12092024-004
PKL file already exists for Caos-12102024-003
PKL file already exists for Caos-12102024-006
PKL file already exists for Diablito-03122024-003
PKL file already exists for Diablito-11122024-003
PKL file already exists for Diablito-11122024-005
PKL file already exists for Diablito-12122024-004
PKL file already exists for Diablito-12122024-006
Loading Alfa-04082020-003 mat file
Time taken: 0.33 seconds for loading mat file
Loading TrialRecord	Warning: No MATLAB class specified for <HDF5 dataset "Angle": shape (5, 1), type "|O"> at path /Trials/TrialRecord/CurrentConditionStimulusInfo/Angle object
done
Loading XY	done
Loading block	done
Loading event03	done
Loading event10	done
Loading eventMarkers	done
Loading eyePupil	done
Loading eyeXY	done
Loading imageName	don

 32%|███▏      | 13/41 [00:08<00:17,  1.61it/s]

Time taken: 0.51 seconds for dumping pkl file [Total: 8.12 seconds]
Loading Alfa-06082020-004 mat file
Time taken: 0.08 seconds for loading mat file
Loading TrialRecord	Warning: No MATLAB class specified for <HDF5 dataset "Angle": shape (5, 1), type "|O"> at path /Trials/TrialRecord/CurrentConditionStimulusInfo/Angle object
done
Loading XY	done
Loading block	done
Loading event03	done
Loading event10	done
Loading eventMarkers	done
Loading eyePupil	done
Loading eyeXY	done
Loading imageName	done
Loading trialStart	done
Loading width	done
Loading words	done
Time taken: 7.62 seconds for loading trials


 34%|███▍      | 14/41 [00:16<00:36,  1.36s/it]

...

Time taken: 0.73 seconds for dumping pkl file [Total: 319.99 seconds]
Loading Beto-14092020-004 mat file
Time taken: 0.06 seconds for loading mat file
Loading TrialRecord	Warning: No MATLAB class specified for <HDF5 dataset "Angle": shape (6, 1), type "|O"> at path /Trials/TrialRecord/CurrentConditionStimulusInfo/Angle object
done
Loading XY	done
Loading block	done
Loading event03	done
Loading event10	done
Loading eventMarkers	done
Loading eyePupil	done
Loading eyeXY	done
Loading imageName	done
Loading trialStart	done
Loading width	done
Loading words	done
Time taken: 9.65 seconds for loading trials


 98%|█████████▊| 40/41 [05:30<00:11, 11.69s/it]

Time taken: 1.06 seconds for dumping pkl file [Total: 330.77 seconds]
Loading Beto-17092020-003 mat file
Time taken: 0.07 seconds for loading mat file
Loading TrialRecord	Warning: No MATLAB class specified for <HDF5 dataset "Angle": shape (5, 1), type "|O"> at path /Trials/TrialRecord/CurrentConditionStimulusInfo/Angle object
done
Loading XY	done
Loading block	done
Loading event03	done
Loading event10	done
Loading eventMarkers	done
Loading eyePupil	done
Loading eyeXY	done
Loading imageName	done
Loading trialStart	done
Loading width	done
Loading words	done
Time taken: 11.91 seconds for loading trials


100%|██████████| 41/41 [05:43<00:00,  8.38s/it]

Time taken: 0.76 seconds for dumping pkl file [Total: 343.52 seconds]


In [105]:
for _, exprow in ExpRecord_Evol.iterrows():
    print(exprow.ephysFN,exprow.Expi)
    data = pkl.load(open(join(pkl_root, f"{exprow.ephysFN}.pkl"), "rb"))
    rasters = data["rasters"]
    meta = data["meta"]
    Trials = data["Trials"]
    imageName = np.squeeze(Trials.imageName)
    stimuli_dir = exprow.stimuli
    break

Caos-12022024-002 1.0


In [117]:
for _, exprow in ExpRecord_Evol.iterrows():
    print(exprow.ephysFN,exprow.Expi)
    data = pkl.load(open(join(pkl_root, f"{exprow.ephysFN}.pkl"), "rb"))
    rasters = data["rasters"]
    meta = data["meta"]
    Trials = data["Trials"]
    imageName = np.squeeze(Trials.imageName)
    stimuli_dir = exprow.stimuli
    prefchan = Trials.TrialRecord.User.prefChan
    prefunit = Trials.TrialRecord.User.unit_perChan
    print(prefchan, prefunit)

Caos-12022024-002 1.0
[[91. 91.]] [[2. 2.]]
Caos-12022024-005 2.0
[[94. 94.]] [[1. 1.]]
Caos-12042024-002 3.0
[[71. 71.]] [[2. 2.]]
Caos-12042024-006 4.0
[[72. 72.]] [[1. 1.]]
Caos-12092024-003 5.0
[[92. 92.]] [[2. 2.]]
Caos-12102024-002 6.0
[[68. 68.]] [[2. 2.]]
Caos-12102024-005 7.0
[[69. 69.]] [[1. 1.]]
Diablito-03122024-002 1.0
[[56. 56.]] [[1. 1.]]
Diablito-11122024-002 2.0
[[42. 42.]] [[2. 2.]]
Diablito-11122024-004 3.0
[[34. 34.]] [[2. 2.]]
Diablito-12122024-003 4.0
[[64. 64.]] [[2. 2.]]
Diablito-12122024-005 5.0
[[50. 50.]] [[1. 1.]]


### Dev zone

In [ ]:
stim_info = []
re_pattern = r'(noise|class)_eig(\d+)_lin([+-]?\d+\.\d+)'
for name in unique_imgnames: 
    match = re.match(re_pattern, name)
    if match:
        space_name = match.groups()[0]
        eig_value = int(match.groups()[1])
        lin_value = float(match.groups()[2])
        stim_info.append({"img_name": name, "space_name": space_name, "eig_id": eig_value, "lin_dist": lin_value, "hessian_img": True, "trial_ids": indices_per_name[name]})
    else:
        stim_info.append({"img_name": name, "space_name": None, "eig_id": None, "lin_dist": None, "hessian_img": False, "trial_ids": indices_per_name[name]})

stim_info_df = pd.DataFrame(stim_info)
